In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Additional libraries
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Torch libraries
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
df = pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")

In [ ]:
# Taking a look at the first 5 rows
df.head()

In [ ]:
# Shape
df.shape

In [ ]:
# Checking for null values
df.isnull().sum()

In [ ]:
# data types and description
df.info()

In [ ]:
# Plotting pairplot
sns.pairplot(df, hue="Outcome")

In [ ]:
# Describe
df.describe()

In [ ]:
# Dividing data into X and y variables
X = df.iloc[:,:-1].values
y = df.iloc[:, -1].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2, test_size=0.2)

In [ ]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)


In [ ]:
# Creating classes for Pytorch model

class ANN(nn.Module):
    def __init__(self, input_features=8, hidden1= 30, hidden2=20, hidden3=20, out_features=2):
        super().__init__()
        self.f_connected1=nn.Linear(input_features,hidden1)
        self.f_connected2=nn.Linear(hidden1,hidden2)
        self.f_connected3=nn.Linear(hidden2,hidden3)
        self.out=nn.Linear(hidden3,out_features)
    def forward(self,x):
        x=F.relu(self.f_connected1(x))
        x=F.relu(self.f_connected2(x))
        x=self.out(x)
        return x
    

In [ ]:
# Instantitiate model
torch.manual_seed(10)

model = ANN()

In [ ]:
model.parameters

In [ ]:
### Backward Propogation-- Loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

In [ ]:

epochs=1000
final_losses=[]
for i in range(epochs):
    i=i+1
    y_pred=model.forward(X_train)
    loss=loss_function(y_pred,y_train)
    final_losses.append(loss)
    if i%10==1:
        print("Epoch number: {} and the loss : {}".format(i,loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
# Plotting the loss over epochs
plt.plot(range(epochs),final_losses)
plt.xlabel("Loss")
plt.ylabel("Epoch")
plt.show()

In [ ]:
#### Prediction In X_test data
predictions=[]
with torch.no_grad():
    for i,data in enumerate(X_test):
        y_pred=model(data)
        predictions.append(y_pred.argmax().item())

print(predictions)

In [ ]:
# Performance of the model
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(y_test,predictions)
cm

In [ ]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,predictions)
score